In [1]:
# import torch
# from transformers import GPT2LMHeadModel
# from transformers import PreTrainedTokenizerFast

In [2]:
# tokenizer = PreTrainedTokenizerFast.from_pretrained("skt/kogpt2-base-v2", bos_token='</s>', eos_token='</s>', unk_token='<unk>', pad_token='<pad>', mask_token='<mask>')
# tokenizer.tokenize("안녕하세요. 한국어 GPT-2 입니다.😤:)l^o")


In [3]:
# model = GPT2LMHeadModel.from_pretrained('skt/kogpt2-base-v2')

# text = '인공지능이란'
# input_ids = tokenizer.encode(text)
# gen_ids = model.generate(torch.tensor([input_ids]),
#                          max_length=128,
#                          repetition_penalty=2.0,
#                          pad_token_id=tokenizer.pad_token_id,
#                          eos_token_id=tokenizer.eos_token_id,
#                          bos_token_id=tokenizer.bos_token_id,
#                          use_cache=True)
# generated = tokenizer.decode(gen_ids[0, :].tolist())
# print(generated)



## Tokenizer

토크나이저는 모델에 대한 입력 준비를 담당합니다. 라이브러리에는 모든 모델에 대한 토크나이저가 포함되어 있습니다.
Tokenize의 Encoding은 각 단어에게 구별되는 고유한 index를 붙여주는 작업을 말합니다. 각 사람에게 고유한 이름이 있듯이, 각 단어에게도 고유한 이름이 있어야 모델이 이를 구별할 수 있습니다. 모델에서는 이 이름이 숫자 인덱스 인 것입니다.

기본 클래스인 `PreTrainedTokenizer` 및 `PreTrainedTokenizerFast` 는 모델에 입력이 되는 문자열 입력을 encoding 합니다.

* 토큰 문자열을 id로 변환하고 역으로 id를 토큰 문자열로 변환 합니다. 인코딩/디코딩 즉, 토큰화 및 정수로 변환하는 함수를 제공합니다.
* 기본 구조(BPE, SentencePiece)와 독립적인 방식으로 어휘에 새 토큰을 추가합니다.
* mask, beginning-of-sentence 등과 같은 특수 토큰을 관리 합니다.

koGPT2 에 입력으로 넣어주는 input_ids 는 tokenizer의 encode 함수에서 자동으로 만들어 줍니다.

모델에 실제로 Input으로 들어가게 되는 Input Ids 와 Label Ids는 ( batch_size, seq_length )의 Shape를 가집니다. 모델에는 Input Ids 가 들어가고 출력된 Logits 과 Label Ids와 비교하게 됩니다. Input Ids 말고 모델에 Input으로 들어가는 것으로 Mask가 있습니다. GPT에서는 다음에 올 토큰을 예측할 때 이전의 토큰들과의 Attention만을 사용해서 예측하기 때문에, 뒤에 오는 토큰들과의 Attention을 구분하기 위해 Future Mask를 사용합니다. Mask 는 특정 토큰을 Attention 에서 구분하기 위해서 사용됩니다. Future Mask는 각각의 토큰이 다른 토큰들에 대해서 Masking의 여부를 나타내는 2차원 행렬입니다.

In [4]:
import numpy as np
import pandas as pd
import torch

from pytorch_lightning import Trainer
from pytorch_lightning.callbacks import model_checkpoint
from pytorch_lightning.core import LightningModule

from torch.utils.data import DataLoader, Dataset

from transformers.optimization import AdamW, get_cosine_schedule_with_warmup
from transformers import PreTrainedTokenizerFast, GPT2LMHeadModel

import re
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


In [5]:
Q_TKN = "<usr>"
A_TKN = "<sys>"
BOS = '</s>'
EOS = '</s>'
MASK = '<unused0>'
SENT = '<unused1>'
PAD = '<pad>'


In [6]:
koGPT2_TOKENIZER = PreTrainedTokenizerFast.from_pretrained('skt/kogpt2-base-v2', bos_token=BOS, eos_token=EOS, unk_token='<unk>', pad_token=PAD, mask_toke=MASK)
model = GPT2LMHeadModel.from_pretrained('skt/kogpt2-base-v2')

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'GPT2Tokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.


In [7]:
charbot_data = pd.read_csv('./ChatbotData.csv')

#test 용으로 300개 데이터만 처리
charbot_data = charbot_data[:300]
charbot_data.head()

,Q,A,label
0,12시 땡!,하루가 또 가네요.,0
1,1지망 학교 떨어졌어,위로해 드립니다.,0
2,3박4일 놀러가고 싶다,여행은 언제나 좋죠.,0
3,3박4일 정도 놀러가고 싶다,여행은 언제나 좋죠.,0
4,PPL 심하네,눈살이 찌푸려지죠.,0


In [8]:
# 파이토치 데이터셋 만들기

class ChatbotDataset(Dataset):
    def __init__(self, chars, max_len = 40): # 데이터셋 전처리
        self._data = chars
        self.max_len = max_len
        self.q_token = Q_TKN
        self.a_token = A_TKN
        self.sent_token = SENT
        self.eos = EOS
        self.mask = MASK
        self.tokenizer = koGPT2_TOKENIZER
    
    def __len__(self): # chatbotdata 의 길이를 리턴
        return len(self._data)

    def __getitem__(self, idx): # 로드한 챗봇 데이터를 차례차례 DataLoader로 넘겨주는 메서드
        turn = self._data.iloc[idx]
        q = turn['Q'] # 질문을 가져온다.
        q = re.sub(r"([?.!,])", r' ',q) # 구둣점들을 제거

        a = turn['A'] # 답변을 가져온다.
        a = re.sub(r"([?.!,])", r' ', a)  # 구둣점들을 제거

        q_token = self.tokenizer.tokenize(self.q_token + q + self.sent_token)
        q_len = len(q_token)

        a_token = self.tokenizer.tokenize(self.a_token + a + self.eos)
        a_len = len(a_token)

        # 질문의 길이가 최대길이보다 크면
        if q_len > self.max_len:
            a_len = self.max_len - q_len # 답변의 길이를 최대길이 - 질문길이
            if a_len <= 0: # 질문의 길이가 너무 길어 질문만으로 최대 길이를 초과 한다면
                q_token = q_token[-(int(self.max_len / 2)) :] # 질문길이를 최대길이의 반으로
                q_len = len(q_token)
                a_len = self.max_len - q_len
            a_token = a_token[:a_len] # 답변의 길이를 최대길이 - 질문길이
            a_len = len(a_token)
        
        # 질문의 길이 + 답변의 길이가 최대길이보다 크면
        if q_len + a_len > self.max_len:
            a_len = self.max_len - q_len # 답변의 길이를 최대길이 - 질문길이
            if a_len <= 0: #질문의 길이가 너무 길어 질문만으로 최대 길이를 초과 한다면
                q_token = q_token[-(int(self.max_len / 2)) :] # 질문길이를 최대길이의 반으로
                q_len = len(q_token)
                a_len = self.max_len - q_len # 답변의 길이를 최대길이 - 질문길이
            a_token = a_token[:a_len]
            a_len = len(a_token)

        # 답변 labels = [mask, mask, ..., mask, ..., <bos>, ...답변... <eos>, <pad> ...]
        labels = [self.mask,] * q_len + a_token[1:]

        # mask = 질문길이 0 + 답변길이 1 + 나머지 0
        mask = [0] * q_len + [1] * a_len + [0] * (self.max_len - q_len - a_len)

        # 답변 labels를 index로 만든다.
        labels_ids = self.tokenizer.convert_tokens_to_ids(labels)

        # 최대길이만큼 padding
        while len(labels_ids) < self.max_len:
            labels_ids += [self.tokenizer.pad_token_id]

        # 질문 + 답변 index 로 만든다.
        token_ids = self.tokenizer.convert_tokens_to_ids(q_token + a_token)

        # 최대길이만큼 padding
        while len(token_ids) < self.max_len:
            token_ids += [self.tokenizer.pad_token_id]

        # 질문+답변, 마스크, 답변
        return (token_ids, np.array(mask), labels_ids)

In [9]:
def collate_batch(batch):
    data = [item[0] for item in batch]
    mask = [item[1] for item in batch]
    label = [item[2] for item in batch]
    return torch.LongTensor(data), torch.LongTensor(mask), torch.LongTensor(label)

In [10]:
train_set = ChatbotDataset(charbot_data, max_len=40)

# 윈도우 환경에서 num_workers 는 무조건 0으로 지정, 리눅스에서는 2
train_dataloader = DataLoader(train_set, batch_size=32, num_workers=0, shuffle=True, collate_fn=collate_batch)
print(device)

cuda


In [11]:
model.to(device)
model.train()

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(51200, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0): GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
      (1): GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dro

In [12]:
learning_rate = 3e-5
criterion = torch.nn.CrossEntropyLoss(reduction='none')
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

epoch = 10
Sneg = -1e18


In [13]:
print("start")
for epoch in range(epoch):
    for batch_idx, samples in enumerate(train_dataloader):
        optimizer.zero_grad()
        token_ids, mask, label = samples
        token_ids, mask, label = token_ids.to(device), mask.to(device), label.to(device)
        out = model(token_ids)
        out = out.logits  # Returns a new tensor with the logit of the elements of input
        mask_3d = mask.unsqueeze(dim=2).repeat_interleave(
            repeats=out.shape[2], dim=2)
        mask_out = torch.where(mask_3d == 1, out, Sneg * torch.ones_like(out))
        loss = criterion(mask_out.transpose(2, 1), label)
        # 평균 loss 만들기 avg_loss[0] / avg_loss[1] <- loss 정규화
        avg_loss = loss.sum() / mask.sum()
        avg_loss.backward()
        # 학습 끝
        optimizer.step()
print("end")


start


C:\Users\aqs45\AppData\Local\Temp\ipykernel_16420\2495599640.py:5: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\torch\csrc\utils\tensor_new.cpp:233.)
  return torch.LongTensor(data), torch.LongTensor(mask), torch.LongTensor(label)


OutOfMemoryError: CUDA out of memory. Tried to allocate 250.00 MiB (GPU 0; 4.00 GiB total capacity; 3.14 GiB already allocated; 0 bytes free; 3.28 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [ ]:
with torch.no_grad():
    while 1:
        q = input("user > ").strip()
        if q == "quit":
            break
        a = ""
        while 1:
            input_ids = torch.LongTensor(koGPT2_TOKENIZER.encode(
                Q_TKN + q + SENT + A_TKN + a)).unsqueeze(dim=0)
            pred = model(input_ids)
            pred = pred.logits
            gen = koGPT2_TOKENIZER.convert_ids_to_tokens(
                torch.argmax(pred, dim=-1).cpu().squeeze().numpy().tolist())[-1]
            if gen == EOS:
                break
            a += gen.replace("▁", " ")
        print("Chatbot > {}".format(a.strip()))
